# Awardee Data
Author: Carson Case [carsonpcase@gmail.com]  
Notebook to analize government contractors and return a csv of Awardees
Download the data here:  
https://sam.gov/data-services/Contract%20Opportunities/Archived%20Data?privacy=Public

In [ ]:
#Import the csvs, ntoe the name has been changed from the default
import pandas as pd

df1 = pd.read_csv("2024data.csv",  encoding = "ISO-8859-1")
df2 = pd.read_csv("2023data.csv",  encoding = "ISO-8859-1")
df = pd.concat([df1,df2])

In [307]:
# parse the Award$ into floats and remove null awardees
df = df[df['Awardee'].apply(lambda x: isinstance(x, str))]
df = df[df['Awardee'] != "null "]

df["Award$"] = df["Award$"].str.replace("$", "").str.replace(",","").str.replace('(Estimated)','').astype(float)

In [308]:
# Count the totals and Award count for each Awardee
totals = pd.DataFrame({
    'Awardee': [], 
    'Total$': [], 
    'ContractCount': []
    })

for index, row in df.iterrows():
    if(not row["Awardee"] in list(totals['Awardee']) and row["Award$"] > 0):
        new_rows = pd.DataFrame({'Awardee': [row["Awardee"]], 'Total$': [0.00], 'ContractCount': [0]})
        totals = pd.concat([totals, new_rows], ignore_index=True)
    if(row["Award$"] > 0):
        prev_value_money = totals[totals['Awardee'] == row["Awardee"]]['Total$']
        prev_value_count = totals[totals['Awardee'] == row["Awardee"]]['ContractCount']
        prev_value_money = (list(prev_value_money)[0])
        prev_value_count = (list(prev_value_count)[0])
        totals[totals['Awardee'] == row["Awardee"]] = [row["Awardee"], prev_value_money + row["Award$"], prev_value_count+1]

totals

,Awardee,Total$,ContractCount
0,"AVIATION PROCUREMENT SYSTEMS, LLC",735900.69,8.0
1,KAMPI COMPONENTS CO INC,5829365.33,66.0
2,TRI-STATE TECHNICAL SALES CORP,275556.00,2.0
3,"Pharmacy Buying Association, Inc. Kansas City ...",62215.76,3.0
4,RIVERSIDE HOME MEDICAL SUPPLY INC Dallas TX 75...,116462.64,1.0
...,...,...,...
5767,FIDELIDADE - COMPANHIA DE SEGUROS SA 1249-001 PT,325912.55,1.0
5768,"STERIS INSTRUMENT MANAGEMENT SERVICES, INC. FO...",312500.04,1.0
5769,TRIDENT RESEARCH LLC TX 78758-7623 USA,28456623.00,1.0
5770,Fort Randall Telephone Company SD 57380-9373 USA,41429.73,1.0


In [309]:
# output to a csv
totals.to_csv("out.csv")